In [ ]:
# The idea here is to create a "loose" LSTM based on lyrics from a wide array of musical artists. The weights
# generated here will be used in transfer learning when training more specifically on one artist
# I think the retraining on the individual artist's lyrics will ideally "overfit" to that one artist's style
# and then it will take less time to do each individual bc the transfer NN will already be made
# That's the idea anyway. %matplotlib inline

In [31]:
%matplotlib inline

import lyricsgenius as lg
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Flatten, Dropout, Embedding, TimeDistributed, Input, InputLayer
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
from keras.utils import np_utils
import pandas as pd
from keras.applications import VGG16
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
# create .txt file with an array of lyrics from determined set of artists

file = open("largeDataSet.txt", "w")
genius = lg.Genius('EVEbdlvJhu4oRGFz56kQIrETSGLVaJDvHkwbNzsyu1ysjU0Jc8x0w641ZqdfXmc8', skip_non_songs=True, 
                   excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)
artists = ["Bob Dylan", "Kanye West", "Taylor Swift", "Kendrick Lamar", "Ariana Grande", "Queen", "Migos", "Frank Ocean",
          "U2", "Neil Young", "BROCKHAMPTON", "Talking Heads", "ABBA", "Billy Joel", "Jack Johnson", "Lorde", "Adele",
          "Pitbull", "*NSYNC", "Vengaboys", "Michael Jackson"]

def getLyrics(arr, k):
    c = 0
    for name in arr:
        try:
            songs = (genius.search_artist(name, max_songs=k, sort='popularity')).songs
            s = [song.lyrics for song in songs]
            file.write("\n \n   <|endoftext|>   \n \n".join(s))
            c += 1
            print(f"Songs grabbed:{len(s)}")
        except:
            print(f"some exception at {name}: {c}")

getLyrics(artists, 5)

Searching for songs by Bob Dylan...

Song 1: "Murder Most Foul"
Song 2: "Blowin’ in the Wind"
Song 3: "The Times They Are A-Changin’"
Song 4: "All Along the Watchtower"
Song 5: "Like a Rolling Stone"

Reached user-specified song limit (5).
Done. Found 5 songs.
Songs grabbed:5
Searching for songs by Kanye West...

Song 1: "Mercy"
Song 2: "Father Stretch My Hands, Pt. 1"
Song 3: "Monster"
Song 4: "Ultralight Beam"
Song 5: "Bound 2"

Reached user-specified song limit (5).
Done. Found 5 songs.
Songs grabbed:5
Searching for songs by Taylor Swift...

Song 1: "​cardigan"
Song 2: "​exile"
Song 3: "Lover"
Song 4: "​the 1"
Song 5: "Look What You Made Me Do"

Reached user-specified song limit (5).
Done. Found 5 songs.
Songs grabbed:5
Searching for songs by Kendrick Lamar...

Song 1: "HUMBLE."
Song 2: "​m.A.A.d city"
Song 3: "Swimming Pools (Drank)"
Song 4: "DNA."
Song 5: "Money Trees"

Reached user-specified song limit (5).
Done. Found 5 songs.
Songs grabbed:5
Searching for songs by Ariana Grande

In [4]:
file = open("/Users/mollykammann/Documents/create_collab/largeDataSet.txt", encoding= "UTF-8")
lyrics = file.read()
lyrics = lyrics.lower()
file.close()
print(lyrics)

'twas a dark day in dallas, november '63
a day that will live on in infamy
president kennedy was a-ridin' high
good day to be livin' and a good day to die
being led to the slaughter like a sacrificial lamb
he said, "wait a minute, boys, you know who i am?"
"of course we do, we know who you are"
then they blew off his head while he was still in the car
shot down like a dog in broad daylight
was a matter of timing and the timing was right
you got unpaid debts, we've come to collect
we're gonna kill you with hatred, without any respect
we'll mock you and shock you and we'll grin in your face
we've already got someone here to take your place
the day they blew out the brains of the king
thousands were watching, no one saw a thing
it happened so quickly, so quick, by surprise
right there in front of everyone's eyes
greatest magic trick ever under the sun
perfectly executed, skillfully done
wolfman, oh wolfman, oh wolfman, howl
rub-a-dub-dub, it's a murder most foul

hush, little children, yo

In [12]:
chars = sorted(list(set(lyrics)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# finding num. of characters in text & number of unique characters
n_characters = len(lyrics)
n_vocabulary = len(chars)

print("Total characters in lyric set: " , n_characters)
print("Total vocabulary in lyric set: " , n_vocabulary)

seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(lyrics) - seqlen - 1, step):
    sentences.append(lyrics[i: i + seqlen + 1])
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1

model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss = 'categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01), 
    metrics=['categorical_crossentropy', 'accuracy'])

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)    

def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(lyrics) - seqlen - 1)
    
    
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = lyrics[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

"""data_x = []
data_y = []



for i in range(0, n_characters - seqlen, 1):
    seq_in = lyrics[i:i+seqlen]
    seq_out = lyrics[i + seqlen]
    data_x.append([char_indices[char] for char in seq_in])
    data_y.append(char_indices[seq_out])
n_patterns = len(data_x)
print( 'Total Patterns : ', n_patterns)

# Reshape X to be suitable to go into LSTM RNN :
X = np.reshape(data_x , (n_patterns, seqlen, 1))
# Normalizing input data :
X = X/ float(n_vocabulary)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)

LSTM_layer_num = 2 # number of LSTM layers
layer_size = [128,128,128,128] # number of nodes in each layer
model = Sequential()
model.add(LSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))
for i in range(1,LSTM_layer_num) :
    model.add(LSTM(layer_size[i], return_sequences=True))
model.add(Flatten())
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.summary()

checkpoint_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]"""



Total characters in lyric set:  239719
Total vocabulary in lyric set:  69
Epoch 1/50
47/47 [==============================] - ETA: 0s - loss: 2.9506 - categorical_crossentropy: 2.9506 - accuracy: 0.2095
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "close your eyes and let the word paint a"
close your eyes and let the word paint and the the gon' the the the ind the the ge the then' the the gong the the the the t the the s cang the gon' the the s the the gon' the the the the d so t you ghang the theng the the the the s the con' in' the the the the ling the the the gong the the me the ge cang the the d me the d the the the gon' ind the me the  ing the me s the gond the the d the gond the the the the the gon' s me the the the
----- diversity: 0.5
----- Generating with seed: "close your eyes and let the word paint a"
close your eyes and let the word paint and yod thin' te it ink be (hand youn te con'  n's thend gome gon' des in'  sget inmisd thang n'

and when i felt the sare the care the streat the streep the mand the same be the streep the stor the same me my sare me (yeah, be me the same the same a steel ster the light stret me (yeah, be the same the place me the light the light the light the sure (hell some a streat the sheel me the same and the streep the strean me (yeah, be me the care the light the sure me the same (yeah)
yeah, be the same all the shere
----- diversity: 0.5
----- Generating with seed: "in twenty lifetimes, i

and when i felt "
in twenty lifetimes, i

and when i felt the care the strees to my live the shill me all sheam life a do like it up, never treet the wanna be free the allorg
the mide (swerve)
she ding see where your mick the puck the ware your boughtate the ling feel it allist do
love the one all the sick to somethy got the feel the pleater the rears in the let sough to leace and see my till for your gring
we deen here the streep change the light your h
----- diversity: 1.0
----- Generating with seed: "

i know the come to go thinkin' off the same
we really day i want to be starting that but we can a tell you see i turn to got some night in could me do
look what you want to be the end to start to be your hood and the light always back of the same and that now you know that the somesting to in (yeah, yeah)
they busting something
i want to be starting se
----- diversity: 1.0
----- Generating with seed: " down
letting the days go by, water flow"
 down
letting the days go by, water flowe

i wann't always drak the nembotills in
wrong mide to 'ervery time

ut chools pad i bet every dead 'ounn times there's a make you, now nothing
kesus alplacters diffenince
pet you never saig tell mallen of this
setr]
i couldn't rearter, i'm two
)
migiticaupe a bad
bulieve amon's that i feel roney'se spy with my dance9)
 like i himbay?"
wishi can in turnsupiffat i'm everything top night down? 
i'm
47/47 [==============================] - 40s 857ms/step - loss: 1.4883 - categorical_crossentropy: 1.4883 - accu

like so, tell my mouts ont
se, i got
i'll say it head, my night in the hightoahe who wifait (what--m-c-je)
mop-p-per)
turnin' man, jouse, my misd chein, that cause i al hin

fridnd, it met mesler, "my new of chin’ slight, ain't no what i'll slow the arto
one of life me down (you're
47/47 [==============================] - 42s 893ms/step - loss: 1.3451 - categorical_crossentropy: 1.3451 - accuracy: 0.5985
Epoch 14/50
47/47 [==============================] - ETA: 0s - loss: 1.3224 - categorical_crossentropy: 1.3224 - accuracy: 0.6058
----- Generating text after Epoch: 13
----- diversity: 0.2
----- Generating with seed: " say "beware doll, you're bound to fall""
 say "beware doll, you're bound to fall"
but i see it was been to be the world a way to be the world i want to be the world do it all the times

we like to party
we like to party
we like to party
we like to party
we like to party
we like to party
we like to party
we like to party
we like to party
we like to party
we like to party


roll–roll–roll my weed on it: the world of the one the summer the truth in the sun
all the color the summer the money to see your fuckin' she suit
pop the one
she said the money to be the one the summer than i see you with the money, i feel the world a feelin' the money and then the morning
so i gone (swerve)
believe, been his willin' like a street my hands (skrrt)
you can talk it like i talk it (walk it), walk it like i talk i
----- diversity: 0.5
----- Generating with seed: "ss-state
roll–roll–roll my weed on it: t"
ss-state
roll–roll–roll my weed on it: the house
when i'm world
the won't really don't wanna feel this toping
so the color the truth in the monning
never sleepin' in the money's gonna give me like i talk it (walk it)
walk it like i talk it (walk it), walk it like i talk it (walk it)
walk it like i talk it (walk it), walk it like i talk it (walk it)
walk it like i talk it (woo)
walk it like i talk it (talk it), walk it like i talk it (wo
----- diversity: 1.0
----- Generati

...

you know how the truth is a wart a fave it had it all at the back when i feet start nobody going dimmin i herver but i think i can fell ace of a dancing is wine to the drank
then the world's been the one (oh oh whoa-oh)
whoa-oh, whoa-oh
whoa-oh
whoa-oh, whoa-oh

take my lash so eazin' bout forever
(ooh dian it ever was a warning some one to get when i heards (rown) where you can't take the beash when i was tr
----- diversity: 1.0
----- Generating with seed: "metimes it hurts instead."

you know how"
metimes it hurts instead."

you know how the not lorcining
and seems 'em back and sing)
we didn't start the's are always be mianings of your licker
macket me first aidee)

well i way i was closs gone
it's a momb-abmy, we're all you of your dayions as the name of love?
i guess we're bing around that get backs
drist me, "beling twore

no, no, bitt like to party
we like to party
we like, but i soon (don't kinke)

i got it (yeah)

back of u
47/47 [==============================] - 39s 837ms/ste

all then this cola! i've on love
who we were young apah
(lot)
see it (white)
two chollie i go? (yeah)
mal it, drack be roling ads, partorin's water flowing up down
i'm the please down
like a song


i, i com boomby, i have home

they down fir my equaies is a manation?
the wrovin' when we we stop, all like a grasple you when with you
you know i'm juit
47/47 [==============================] - 40s 842ms/step - loss: 1.0993 - categorical_crossentropy: 1.0993 - accuracy: 0.6694
Epoch 40/50
47/47 [==============================] - ETA: 0s - loss: 1.0980 - categorical_crossentropy: 1.0980 - accuracy: 0.6696
----- Generating text after Epoch: 39
----- diversity: 0.2
----- Generating with seed: "are
you're wonderful so far, and it's mo"
are
you're wonderful so far, and it's more through to go
i'm gonna wish you okay?
so, annie, are you okay?
so, annie, are you okay? are you okay, annie?
annie, are you okay? are you okay, annie?
annie, are you okay? are you okay, annie?
annie, are you okay? are y

but when i dream, i'm so thers and the truth in the street me (yeah, yeah)
you're stuck in the middle (yeah, yeah)
you're stuck in the middle (yeah, yeah)
you're stuck in the middle (yeah, yeah)
you're stuck in the middle (yeah, yeah)
you're stuck in the middle (yeah, yeah)
you're stuck in the middle (yeah, yeah)
you're stuck in the middle (yeah, yeah)
you're stuck in the mountin' with me

the way that you do a warnin
----- diversity: 0.5
----- Generating with seed: "ey see men fallin'
but when i dream, i'm"
ey see men fallin'
but when i dream, i'm still like it's stir fry (whip it)
in the kitchen, watch compen the lived anymore
i hate to say your could i was someone who every vacat night to get me come and he many his with your backin' but it like it's stir fry (whip it)
in tire in the carr
man in the mind full off of you seen that you can't i've been some going in your hearts is a weepin' and sunder
to go and get your kinds
she'll say is 
----- diversity: 1.0
----- Generating with se

swing your partner 'round and real shalala, shalala in the man if you see, you know when you in the change he motel room
you pin't heard the taught it all the street
you know when you really real back ean just like home

fuck of love
we and the same of livin'
hey, hey
now that you can't come back to me, kick me, it could be all this more
it was make your girl, same as it everybody don't make a life times in the scarse
----- diversity: 1.0
----- Generating with seed: "she will (timber)

swing your partner 'r"
she will (timber)

swing your partner 'racking out your best every dayes tooinds
last willin' down?
they you know where taukhour doplo got man)
conke
i duss you gon' up
lap are bass in broight uh

hey, hey, n've been the took (yeah, rmash)

thris bid this is what you can't tell flowsicoraza sharon, in lord, nigga, i wake a gold chain on some wheel, baby-cookin' niggas you don't know
you spouciup up that na some what your piece
winis ano
47/47 [==============================] - 42s 

"data_x = []\ndata_y = []\n\n\n\nfor i in range(0, n_characters - seqlen, 1):\n    seq_in = lyrics[i:i+seqlen]\n    seq_out = lyrics[i + seqlen]\n    data_x.append([char_indices[char] for char in seq_in])\n    data_y.append(char_indices[seq_out])\nn_patterns = len(data_x)\nprint( 'Total Patterns : ', n_patterns)\n\n# Reshape X to be suitable to go into LSTM RNN :\nX = np.reshape(data_x , (n_patterns, seqlen, 1))\n# Normalizing input data :\nX = X/ float(n_vocabulary)\n# One hot encode the output targets :\ny = np_utils.to_categorical(data_y)\n\nLSTM_layer_num = 2 # number of LSTM layers\nlayer_size = [128,128,128,128] # number of nodes in each layer\nmodel = Sequential()\nmodel.add(LSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))\nfor i in range(1,LSTM_layer_num) :\n    model.add(LSTM(layer_size[i], return_sequences=True))\nmodel.add(Flatten())\nmodel.add(Dense(y.shape[1]))\nmodel.add(Activation('softmax'))\nmodel.compile(loss = 'categorical_crossent

In [ ]:
"""# Fit the model :
model_params = {'epochs':30,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}

model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])"""

In [6]:
"""weights = './Weights-LSTM-improvement-030-0.88942-bigger.hdf5' # weights file path
model.load_weights(weights)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')"""

ValueError: Shapes (12800, 71) and (12800, 69) are incompatible

In [31]:
"""# set a random seed :
start = np.random.randint(0, len(data_x)-1)
pattern = data_x[start]
print('Seed : ')
print("\"",''.join([indices_char[value] for value in pattern]), "\"\n")

# How many characters you want to generate
generated_characters = 300

# Generate Charachters :
for i in range(generated_characters):
    x = np.reshape(pattern, ( 1, len(pattern), 1))
    x = x / float(n_vocabulary)
    prediction = model.predict(x,verbose = 0)
    index = np.argmax(prediction)
    result = indices_char[index]
    #seq_in = [int_chars[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('\nDone')"""

Seed : 
" vin' in a mansion in all your raps, though
but your shit look like the trap on this google maps, tho "

ugh
wh bl wht trm they ber't no worne, yhoh h'm mnver ioong
wied it notsillg ckg i wicn the simghs bnlwnir

hrmpi to hake and on thi srrsonnss
inne nn bid oond ll toint on, you donne i mad to foay mi ponp you goon iow yhul ltr 
you kn noww wot' tto?  
iomsrnno'
toes ohnh, iuo ifnd wher you iaven wo 
Done


In [17]:
"""def generateLyricsOneArtist(artist):
    newFileName = str(artist) + ".txt"
    file = open(newFileName, "w")
    genius = lg.Genius('EVEbdlvJhu4oRGFz56kQIrETSGLVaJDvHkwbNzsyu1ysjU0Jc8x0w641ZqdfXmc8', skip_non_songs=True, 
                   excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)
    artists = [artist]

    def getLyrics(arr, k):
        c = 0
        for name in arr:
            try:
                songs = (genius.search_artist(name, max_songs=k, sort='popularity')).songs
                s = [song.lyrics for song in songs]
                file.write("\n \n      \n \n".join(s))
                c += 1
                print(f"Songs grabbed:{len(s)}")
            except:
                print(f"some exception at {name}: {c}")

    getLyrics(artists, 2)
    file = open(newFileName, "r")
    lyrics = file.read()
    lyrics = lyrics.lower()
    print(lyrics)
    chars = sorted(list(set(lyrics)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    # finding num. of characters in text & number of unique characters
    n_characters = len(lyrics)
    n_vocabulary = len(chars)

    print("Total characters in lyric set: " , n_characters)
    print("Total vocabulary in lyric set: " , n_vocabulary)

    seqlen = 100
    data_x = []
    data_y = []

    for i in range(0, n_characters - seqlen, 1):
        seq_in = lyrics[i:i+seqlen]
        seq_out = lyrics[i + seqlen]
        data_x.append([char_indices[char] for char in seq_in])
        data_y.append(char_indices[seq_out])
    n_patterns = len(data_x)
    print( 'Total Patterns : ', n_patterns)

    # Reshape X to be suitable to go into LSTM RNN :
    X = np.reshape(data_x , (n_patterns, seqlen, 1))
    # Normalizing input data :
    X = X/ float(n_vocabulary)
    # One hot encode the output targets :
    y = np_utils.to_categorical(data_y)
    
    LSTM_layer_num = 2 # number of LSTM layers
    layer_size = [128,128,128,128] # number of nodes in each layer
    Indepmodel = Sequential()
    Indepmodel.add(LSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))
    for i in range(1,LSTM_layer_num) :
        Indepmodel.add(LSTM(layer_size[i], return_sequences=True))
    Indepmodel.add(Flatten())
    Indepmodel.add(Dense(y.shape[1]))
    Indepmodel.add(Activation('softmax'))
    Indepmodel.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    Indepmodel.summary()
    
    
    


generateLyricsOneArtist("U2")"""


Searching for songs by U2...

Song 1: "One"
Song 2: "With or Without You"

Reached user-specified song limit (2).
Done. Found 2 songs.
Songs grabbed:2
is it getting better
or do you feel the same?
will it make it easier on you, now
you got someone to blame?

you say one love, one life
when it's one need in the night
one love, we get to share it
leaves you, baby, if you don't care for it

did i disappoint you
or leave a bad taste in your mouth?
you act like you never had love
and you want me to go without

well, it's too late tonight
to drag the past out into the light
we're one, but we're not the same
we get to carry each other, carry each other
one

have you come here for forgiveness?
have you come to raise the dead?
have you come here to play jesus
to the lepers in your head?

did i ask too much? more than a lot
you gave me nothing, now it's all i got
we're one, but we're not the same
well, we hurt each other, then we do it again

you say love is a temple, love a higher law
love is a

In [19]:
for diversity in [0.5]:
    print('----- diversity:', diversity)

    generated = ''
    sentence = lyrics[1486:1526]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(200):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.


        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)



----- diversity: 0.5
----- Generating with seed: " say dallas don't love you, mr. presiden"
 say dallas don't love you, mr. president
you know that you can't might
i have not the greater plocket (woo, hah)
walk it like i talk it, walk it, walk it like i talk it (walk it), walk it like i talk it (walk it)
walk it like i talk it (ta


In [20]:
specificModel = keras.models.clone_model(
    model, input_tensors=None, clone_function=None
)

specificModel.set_weights(model.get_weights())

#test
for diversity in [0.5]:
    print('----- diversity:', diversity)

    generated = ''
    sentence = lyrics[1486:1526]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(200):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.


        preds = specificModel.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)




----- diversity: 0.5
----- Generating with seed: " say dallas don't love you, mr. presiden"
 say dallas don't love you, mr. presidention my hand
this is burning shatel
boom, boom, boom, boom!!
i wanna go boot, babe (swerve)

well in my dead on your love if you like to party
get on and move your fuckin' hanis (wish it)

ooh-ooh-ooh


In [34]:
#get one artist's lyrics from genius into a txt file to retrain the network 


def createSpecificModel(artistName):
    title = artistName.replace(" ", "") + ".txt"
    file = open(title, "w")
    genius = lg.Genius('EVEbdlvJhu4oRGFz56kQIrETSGLVaJDvHkwbNzsyu1ysjU0Jc8x0w641ZqdfXmc8', skip_non_songs=True, 
                   excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)
    artists = [artistName]
    getLyrics(artists, 1)
    file = open(title, encoding = "UTF-8")
    specificLyrics = file.read()
    specificLyrics = lyrics.lower()
    file.close()
    print(specificLyrics)
    
    chars = sorted(list(set(specificLyrics)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    # finding num. of characters in text & number of unique characters
    n_characters = len(specificLyrics)
    n_vocabulary = 69 #len(chars)

    print("Total characters in lyric set: " , n_characters)
    print("Total vocabulary in lyric set: " , n_vocabulary)

    seqlen = 40
    step = seqlen
    sentences = []
    for i in range(0, len(specificLyrics) - seqlen - 1, step):
        sentences.append(specificLyrics[i: i + seqlen + 1])
    x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
            x[i, t, char_indices[char_in]] = 1
            y[i, t, char_indices[char_out]] = 1
            
            
            
    
            
    specificModel = keras.models.clone_model(
    model, input_tensors=None, clone_function=None)

    specificModel.set_weights(model.get_weights())
    

    specificModel.add(InputLayer(input_shape = (None, 40, 68)))

    
    specificModel.compile(
        loss = 'categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.01), 
        metrics=['categorical_crossentropy', 'accuracy'])
    
    
    
    
    
    
    
    
    
    
    
    def sample(preds, temperature=1.0):
        """Helper function to sample an index from a probability array."""
        preds = np.asarray(preds).astype('float64')
        preds = np.exp(np.log(preds) / temperature)  # softmax
        preds = preds / np.sum(preds)                #
        probas = np.random.multinomial(1, preds, 1)  # sample index
        return np.argmax(probas)    

    def on_epoch_end(epoch, _):
        """Function invoked at end of each epoch. Prints generated text."""
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(specificLyrics) - seqlen - 1)


        for diversity in [0.2, 0.5, 1.0]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = specificLyrics[start_index: start_index + seqlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, seqlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.


                preds = model.predict(x_pred, verbose=0)
                next_index = sample(preds[0, -1], diversity)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    specificModel.fit(x, y,
              batch_size=128,
              epochs=10,
              callbacks=[print_callback])

createSpecificModel("Kanye West")


Searching for songs by Kanye West...

Song 1: "Mercy"

Reached user-specified song limit (1).
Done. Found 1 songs.
some exception at Kanye West: 0
well, it is a weepin' and a moanin' and a gnashin' of teeth
it is a weepin' and a mournin' and a gnashin' of teeth
it is a—when it comes to my sound which is the champion sound
believe, believe

o-o-o-o-o-okay, lamborghini mercy
your chick, she so thirsty
i-i-i-i-i'm in that two-seat lambo
with your girl, she tryna jerk me (believe)
o-o-o-o-o-okay, lamborghini mercy
your chick, she so thirsty
i-i-i-i-i'm in that two-seat lambo
with your girl, she tryna jerk me
o-o-o-o-o-okay, lamborghini mercy (swerve)
your chick, she so thirsty (swerve)
i-i-i-i-i'm in that two-seat lambo
with your girl, she tryna jerk me (woah, believe)
o-o-o-o-o-okay, lamborghini mercy
your chick, she so thirsty (boy)
i-i-i-i-i'm in that two-seat lambo (boy)
with your girl, she tryna jerk me

okay, drop it to the floor, make that ass shake (shake, shake)
woah, make the gro

Epoch 1/10


ValueError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib